In [1]:
# 05_elasticity_analysis.ipynb
# │
# ├── 1. Imports & configuration
# ├── 2. Data preparation for elasticity
# ├── 3. Price–Quantity relationship (EDA)
# ├── 4. Elasticity estimation
# ├── 5. Elasticity by product / category
# ├── 6. Sanity checks & business interpretation
# └── 7. Outputs for pricing optimization


In [2]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pandas as pd
import matplotlib.pyplot as plt

from src.elasticity.elasticity import *


In [3]:
df = pd.read_csv(
    "../data/processed/indian_machinery_transaction_data_3.csv",
    parse_dates=["invoice_date"]
)

df.head()

,invoice_id,invoice_date,year,month,season,customer_type,brand,product_name,product_category,specification,quantity,mrp,selling_price,unit_cost,discount_pct,revenue,profit
0,500001,2019-01-01,2019,1,Winter,Retail,Supreme,PVC Pipe,Pipe,0.5 inch,1,89.61,83.49,70.69,6.83,83.49,12.80
1,500061,2019-01-01,2019,1,Winter,Retail,Jain,Sprinkler Set,Irrigation,Heavy Duty,2,2257.78,2224.77,1902.32,1.46,4449.54,644.90
2,500060,2019-01-01,2019,1,Winter,Contractor,Jain,Sprinkler Set,Irrigation,Mini,2,1446.31,1334.39,1222.70,7.74,2668.78,223.38
3,500059,2019-01-01,2019,1,Winter,Farmer,Jain,Sprinkler Set,Irrigation,Mini,1,1446.31,1195.99,1021.78,17.31,1195.99,174.21
4,500058,2019-01-01,2019,1,Winter,Retail,Jain,Sprinkler Set,Irrigation,Mini,3,1446.31,1423.58,1200.86,1.57,4270.74,668.16


In [4]:
results_df, weekly_df = elasticity_pipeline_with_models(
    df=df,
)


In [6]:
(
    results_df
    .sort_values(["product_id", "test_mape"])
    .groupby("product_id")
    .head(3)
)


,product_id,model,test_mape,n_train_weeks,n_test_weeks,price_elasticity
5,cpvc_pipe__1_5_inch,GradientBoosting,14.855412,270,67,NaN
4,cpvc_pipe__1_5_inch,RandomForest,14.917090,270,67,NaN
0,cpvc_pipe__1_5_inch,OLS,15.055364,270,67,-0.270416
6,cpvc_pipe__1_inch,OLS,16.217450,270,67,-0.060370
7,cpvc_pipe__1_inch,Ridge,16.507682,270,67,0.016765
...,...,...,...,...,...,...
186,sprinkler_set__mini,OLS,16.172699,270,67,-0.564986
187,sprinkler_set__mini,Ridge,16.469462,270,67,-0.091041
194,thread_tape__standard,Lasso,14.906318,270,67,-0.000000
195,thread_tape__standard,ElasticNet,15.232138,270,67,-0.000000


In [7]:
best_models = (
    results_df
    .sort_values("test_mape")
    .groupby("product_id")
    .first()
    .reset_index()
)

best_models


,product_id,model,test_mape,n_train_weeks,n_test_weeks,price_elasticity
0,cpvc_pipe__1_5_inch,GradientBoosting,14.855412,270,67,-0.270416
1,cpvc_pipe__1_inch,OLS,16.217450,270,67,-0.060370
2,cpvc_pipe__2_inch,RandomForest,11.941914,270,67,0.012885
3,drip_irrigation_kit__0_5_acre,RandomForest,14.932386,270,67,-0.221562
4,drip_irrigation_kit__1_acre,RandomForest,14.892897,270,67,-0.349064
5,drip_irrigation_kit__2_acre,RandomForest,14.771788,270,67,-0.000000
6,foot_valve__1_inch,Lasso,15.009726,270,67,-0.000000
7,motor_pump__0_5_hp_single_phase,Ridge,15.088422,270,67,-0.022458
8,motor_pump__1_5_hp_single_phase,GradientBoosting,14.233244,270,67,-0.034159
9,motor_pump__1_hp_single_phase,RandomForest,17.027625,270,67,-0.433106


In [8]:
best_models[[
    "product_id",
    "model",
    "price_elasticity",
    "test_mape"
]]


,product_id,model,price_elasticity,test_mape
0,cpvc_pipe__1_5_inch,GradientBoosting,-0.270416,14.855412
1,cpvc_pipe__1_inch,OLS,-0.060370,16.217450
2,cpvc_pipe__2_inch,RandomForest,0.012885,11.941914
3,drip_irrigation_kit__0_5_acre,RandomForest,-0.221562,14.932386
4,drip_irrigation_kit__1_acre,RandomForest,-0.349064,14.892897
5,drip_irrigation_kit__2_acre,RandomForest,-0.000000,14.771788
6,foot_valve__1_inch,Lasso,-0.000000,15.009726
7,motor_pump__0_5_hp_single_phase,Ridge,-0.022458,15.088422
8,motor_pump__1_5_hp_single_phase,GradientBoosting,-0.034159,14.233244
9,motor_pump__1_hp_single_phase,RandomForest,-0.433106,17.027625


Negative elasticities indicate demand sensitivity to price increases.
Lagged price effects capture delayed customer response,
common in durable and machinery products.


- ElasticNet offers the best balance between interpretability and accuracy
- Lagged price and demand significantly improve model stability
- Tree models outperform on prediction but lack economic interpretability
